In [ ]:
%load_ext autoreload
%autoreload 2

import os, sys, time
import numpy as np
import scipy as sp
import pandas as pd
import intake
    
from pathlib import Path
from pprint import pprint as pp
p = print 

from sklearn.externals import joblib
import pdb

import matplotlib.pyplot as plt
%matplotlib inline

# ignore warnings
import warnings
if not sys.warnoptions:
    warnings.simplefilter('ignore')
    
# Don't generate bytecode
sys.dont_write_bytecode = True

In [ ]:
import holoviews as hv
import xarray as xr

from holoviews import opts
from holoviews.operation.datashader import datashade, shade, dynspread, rasterize
from holoviews.streams import *
from holoviews import streams
import geoviews as gv
import geoviews.feature as gf
from geoviews import tile_sources as gvts


import geopandas as gpd
import cartopy.crs as ccrs
import cartopy.feature as cf

import panel as pn
hv.notebook_extension('bokeh')
hv.Dimension.type_formatters[np.datetime64] = '%Y-%m-%d'
pn.extension()

In [ ]:
# Add the utils directory to the search path
UTILS_DIR = Path('../utils').absolute()
assert UTILS_DIR.exists()
if str(UTILS_DIR) not in sys.path:
    sys.path.insert(0, str(UTILS_DIR))
    print(f"Added {str(UTILS_DIR)} to sys.path")

pp(sys.path)
    

In [ ]:
from utils import get_mro as mro, nprint
import utils as u

In [ ]:
# Grab registered bokeh renderer
print("Currently available renderers: ", *hv.Store.renderers.keys())
renderer = hv.renderer('bokeh')

To deinf a curve on a plane (ie. planary curve) we need
- parameter, eg. p $\in [0,1]$
- two functions $x(p)$ and $y(p)$, which define the coordinate of the point $C(p)$ in x and y axis, respectively

In [ ]:
class Curve(param.Parameterized):
    p = param.Number(default=0.0, bounds=(0,1))
    
    def __init__(self, xfunc=None, yfunc=None):
        self.xfunc= xfunc or (lambda p: np.sin(2*np.pi*p))
        self.yfunc = yfunc or (lambda p: np.cos(2*np.pi*p))

        self.overview = self.get_overview()
        self.x = self.xfunc(self.p)
        self.y = self.yfunc(self.p)
        self.dmap_point = hv.Dynamic(hv.Points( [(self.x, self.y)], kdims=['x','y']))

        
    @param.depends('p')
    def update_xy(self):
        print('p changed and get_xy is called')
        self.x = self.xfunc(self.p)
        self.y = self.yfunc(self.p)
        self.hv_point = hv
        
    def get_overview(self):
        xs = [self.xfunc(p) for p in np.linspace(0,1,100)]
        ys = [self.yfunc(p) for p in np.linspace(0,1,100)]
        overview = hv.Points( (xs,ys), ['x','y'] ).opts(padding=0.1)
#         display(points)
        return overview
    
    def view(self):
        display(self.overview * hv.

In [ ]:
curve = Curve()

In [ ]:
curve.show()

In [ ]:
PStream = Stream.define('pstream', p=0.0, bounds=(0,1))

In [ ]:
pstream = PStream()

In [ ]:
pstream.print_param_values()

In [ ]:
pstream.event(p=2.)

In [ ]:
#using `pipe` to push data to the visualization
## 1. setup a dynamicmap with empty element 
pipe = Pipe(data=[])
dmap_points = hv.DynamicMap(hv.Points,streams=[pipe])
dmap_points.opts(
    opts.Points(color='green', 
                size=5,
                xlim=(-1,1), ylim=(-1,1),width=500, height=500)
).opts(padding=0.1);


## Curve Generator

In [ ]:
def gen_curve(cfunc, n_steps=100):
    p = 0
    dp = 1./n_steps
    count = 0
    while p<=1:
        x,y = cfunc(p)
        p, curr_p = p+dp, p
        count, curr_count = count+1, count
        yield pd.DataFrame([(curr_count, curr_p, x, y)], columns=['count', 'p', 'x','y'])
        

In [ ]:
n = 100
ps = np.linspace(0,1,num=n)

# define curve function
xfunc = lambda p: np.sin(2*np.pi*p)
yfunc = lambda p: np.cos(2*np.pi*p)
cfunc = lambda p: (xfunc(p), yfunc(p))

In [ ]:
# another function
xfunc = lambda p: np.sin(2*np.pi*p)
yfunc = lambda p: np.sin(2*np.pi*p)
cfunc = lambda p: (xfunc(p), yfunc(p))

In [ ]:
# another function
xfunc = lambda p: np.sin(4*np.pi*p)
yfunc = lambda p: np.cos(2*np.pi*p)
cfunc = lambda p: (xfunc(p), yfunc(p))

In [ ]:
cfunc(0), cfunc(1)

In [ ]:
g_curve = gen_curve(cfunc, n)
next(g_curve)
# 

---
## Method1: Buffer to push data
Push data  using pipe or buffer

In [ ]:
# Define buffer 
example = pd.DataFrame({'count': [], 'p': [], 'x':[], 'y':[]})
dfstream = Buffer(example, length=100, index=False)

In [ ]:
## visualization components
H,W = 500,500
dmap_points = hv.DynamicMap(
    lambda data: hv.Points(data, kdims=['x','y'], group='Curve'),
    streams=[dfstream]).opts(color='p')
curve_opts = opts.Points(size=5,xlim=(-1,1), ylim=(-1,1),width=W, height=H)
dmap_points.opts(curve_opts);
# dmap_points

In [ ]:
dmap_x = hv.DynamicMap(
    lambda data: hv.Points( data, kdims=['p','x'], group='XCoord'),
    streams=[dfstream]).opts(color='p')
xopts = opts.Points('XCoord', width=W, height=H, size=5, xlim=(0,1), 
                    padding=0.1, invert_axes=True, invert_yaxis=True)

dmap_x.opts(xopts);
# dmap_x

In [ ]:
dmap_y = hv.DynamicMap(
    lambda data: hv.Points( data, kdims=['p','y'], group='YCoord'),
    streams=[dfstream]).opts(color='p')
yopts = opts.Points('YCoord', width=W, height=H, size=5, xlim=(0,1), padding=0.1, invert_xaxis=True)
dmap_y.opts(yopts);
# dmap_y


In [ ]:
(
    dmap_points + dmap_y
    + dmap_x
).cols(2)

In [ ]:
# send data through the buffer
dfstream.clear()
n = 100
g_curve = gen_curve(cfunc, n)
for i in range(n):
    dfstream.send(next(g_curve))
    time.sleep(0.3)

        

---
## Method2: Use `streamz.dataframe` and `hv.streams.Buffer`

In [ ]:
import streamz
import streamz.dataframe

In [ ]:
sdf = streamz.dataframe.DataFrame(example=example)

In [ ]:
sdf.example


In [ ]:
next(gen_curve())


In [ ]:
dmap2 = hv.DynamicMap(
    lambda data: hv.Points(data, kdims=['x','y'], group='Curve'),
    streams=[Buffer(sdf)]).opts(color='p')
# dmap2

In [ ]:
dmap2_x = hv.DynamicMap(
    lambda data: hv.Points( data, kdims=['p','x'], group='XCoord'),
    streams=[Buffer(sdf)]).opts(color='p')

xopts = opts.Points('XCoord', width=W, height=H, size=5, xlim=(0,1), 
                    padding=0.1, invert_axes=True, invert_yaxis=True)

dmap2_x.opts(xopts);
# dmap_x

In [ ]:
dmap2_y = hv.DynamicMap(
    lambda data: hv.Points( data, kdims=['p','y'], group='YCoord'),
    streams=[Buffer(sdf)]).opts(color='p')
yopts = opts.Points('YCoord', width=W, height=H, size=5, xlim=(0,1), padding=0.1)
dmap_y.opts(yopts);

In [ ]:
g_curve = gen_curve()
for i in range(100):
    sdf.emit(next(g_curve))

In [ ]:
(dmap2 + dmap2_y +dmap2_x).cols(2)

## Let's encaptulate the simulation process, given a curve generator 


In [ ]:
def get_new_plots(dfstream):
    
    ## visualization components
    H,W = 500,500
    dmap_points = hv.DynamicMap(
        lambda data: hv.Points(data, kdims=['x','y'], group='Curve'),
        streams=[dfstream]).opts(color='p')
    
    curve_opts = opts.Points(size=5,xlim=(-1,1), ylim=(-1,1),width=W, height=H)
    dmap_points.opts(curve_opts);
    # dmap_points
    
    
    dmap_x = hv.DynamicMap(
    lambda data: hv.Points( data, kdims=['p','x'], group='XCoord'),
    streams=[dfstream]).opts(color='p')
    xopts = opts.Points('XCoord', width=W, height=H, size=5, xlim=(0,1), 
                        padding=0.1, invert_axes=True, invert_yaxis=True)

    dmap_x.opts(xopts);
    
    dmap_y = hv.DynamicMap(
    lambda data: hv.Points( data, kdims=['p','y'], group='YCoord'),
    streams=[dfstream]).opts(color='p')
    yopts = opts.Points('YCoord', width=W, height=H, size=5, xlim=(0,1), padding=0.1, invert_xaxis=True)
    dmap_y.opts(yopts);

    overlay = (dmap_points + dmap_y + dmap_x).cols(2)
    return overlay

### Setup for the simulation

In [ ]:
# Define buffer 
example = pd.DataFrame({'count': [], 'p': [], 'x':[], 'y':[]})
dfstream = Buffer(example, length=100, index=False)

# Define curve function
n_steps = 100
a,b = 1,0.5
xfunc = lambda p: (a-b)*np.cos(p) + b*np.cos((a/b-1)*p)
yfunc = lambda p: (a-b)*np.sin(p) - b*np.sin((a/b-1)*p)
cfunc = lambda p: (xfunc(p), yfunc(p))
g_curve = gen_curve(cfunc, n_steps)


# run
# get a new overlay
display(get_new_plots(dfstream))


# send data through the buffer
dfstream.clear()
for i in range(n_steps):
    dfstream.send(next(g_curve))
    time.sleep(0.3)


Try the simulation with various curve equations
- [fifty famous curves](https://elepa.files.wordpress.com/2013/11/fifty-famous-curves.pdf)

## Let's make a simulator class
- [important reference](http://holoviews.org/user_guide/Dashboards.html)
- [Linking pn.widgets and holoviews plots](https://panel.pyviz.org/user_guide/Links.html)

In [ ]:
def get_new_plots(dfstream):
    
    ## visualization components
    H,W = 500,500
    dmap_points = hv.DynamicMap(
        lambda data: hv.Points(data, kdims=['x','y'], group='Curve'),
        streams=[dfstream]).opts(color='p')
    
    curve_opts = opts.Points(size=5,xlim=(-1,1), ylim=(-1,1),width=W, height=H)
    dmap_points.opts(curve_opts);
    # dmap_points
    
    
    dmap_x = hv.DynamicMap(
    lambda data: hv.Points( data, kdims=['p','x'], group='XCoord'),
    streams=[dfstream]).opts(color='p')
    xopts = opts.Points('XCoord', width=W, height=H, size=5, xlim=(0,1), 
                        padding=0.1, invert_axes=True, invert_yaxis=True)

    dmap_x.opts(xopts);
    
    dmap_y = hv.DynamicMap(
    lambda data: hv.Points( data, kdims=['p','y'], group='YCoord'),
    streams=[dfstream]).opts(color='p')
    yopts = opts.Points('YCoord', width=W, height=H, size=5, xlim=(0,1), padding=0.1, invert_xaxis=True)
    dmap_y.opts(yopts);

    overlay = (dmap_points + dmap_y + dmap_x).cols(2)
    return overlay

In [ ]:
import streamz
import streamz.dataframe
class Simulator(param.Parameterized):
    
    ################################################################################
    # Instance Parameters
    ################################################################################
    a = param.Number(default=1.0)
    b = param.Number(default=0.25)
    to_run_clean = param.Boolean(False, doc='Simulation run boolean parameter')
    
    
    ################################################################################
    # Constant class properties
    ################################################################################
    H,W = 500,500
    curve_opts = opts.Points(size=5,xlim=(-1,1), ylim=(-1,1),width=W, height=H)
    xopts = opts.Points('XCoord', width=W, height=H, size=5, xlim=(0,1), 
                        padding=0.1, invert_axes=True, invert_yaxis=True)
    yopts = opts.Points('YCoord', width=W, height=H, size=5, xlim=(0,1), padding=0.1, invert_xaxis=True)

    
    ################################################################################
    # Initialization
    ################################################################################
    def __init__(self, n_steps=100, **kwargs):
        super().__init__(**kwargs)

        self.count = defaultdict(int)
        self.n_steps = n_steps
        self.example = pd.DataFrame({'count': [], 'p': [], 'x':[], 'y':[]})
        self.data_src = streamz.dataframe.DataFrame(example=self.example)
        self.dfstream = Buffer(self.data_src, length=100, index=False)
        self.set_dmap_curve()
        self.set_dmap_x()
        self.set_dmap_y()
        self.overlay = (self.dmap_curve + self.dmap_y + self.dmap_x).cols(2)
        
        self.set_funcs()

    def set_dmap_curve(self):
        self.count['set_curve'] += 1

        dmap_curve = hv.DynamicMap(
            lambda data: hv.Points(data, kdims=['x','y'], group='Curve'),
            streams=[self.dfstream]).opts(color='p')
        self.dmap_curve = dmap_curve.opts(self.curve_opts);

    def set_dmap_x(self):
        self.count['set_x'] += 1
        dmap_x = hv.DynamicMap(
            lambda data: hv.Points( data, kdims=['p','x'], group='XCoord'),
            streams=[self.dfstream]).opts(color='p')
        self.dmap_x = dmap_x.opts(self.xopts)
        
    def set_dmap_y(self):
        self.count['set_y'] += 1
        dmap_y = hv.DynamicMap(
            lambda data: hv.Points( data, kdims=['p','y'], group='YCoord'),
            streams=[self.dfstream]).opts(color='p')
        self.dmap_y = dmap_y.opts(self.yopts)
        
    @param.depends('a','b',watch=True)
    def set_funcs(self):
        print('set_funcs called')
        self.count['set_funcs'] += 1
        self.xfunc = lambda p: (self.a-self.b)*np.cos(p) + self.b*np.cos((self.a/self.b-1)*p)
        self.yfunc = lambda p: (self.a-self.b)*np.sin(p) - self.b*np.sin((self.a/self.b-1)*p)
        self.cfunc = lambda p: (self.xfunc(p), self.yfunc(p))
        
    ################################################################################
    # Holoviews components
    ################################################################################
    @param.depends('a','b','to_run_clean', watch=True)
    def get_count(self):
        df = pd.DataFrame(self.count.items(), columns=['name', 'count'])
        return hv.Table(df)
    

    @param.depends('to_run_clean', watch=True)
    def run_clean(self):
        # send data through the buffer
        self.count['run'] += 1
        self.dfstream.clear()

        if self.to_run_clean:
            curve_generator = gen_curve(self.cfunc, self.n_steps)

            for i in range(self.n_steps):
                self.data_src.emit(next(curve_generator))
                time.sleep(0.1)
        
        
    ################################################################################
    # Set DynamicMaps for linking the plots with parameters
    ################################################################################
    def view(self):
        return pn.panel(self.overlay)

    def dyn_count(self):
        return pn.panel(hv.DynamicMap(self.get_count))

    

In [ ]:
sim = Simulator()
# pn.panel( sim.view())

In [ ]:
pn.Column(
    pn.Row(pn.WidgetBox(sim.param), sim.view()),
    pn.panel(sim.dyn_count())
)

---
### Animation with `play` and `stop`


In [ ]:
from bokeh.models import Slider, Button
from bokeh.layouts import layout
from bokeh.io import curdoc
from bokeh.plotting import show

In [ ]:
curve_gen = gen_curve(cfunc)
curve_gen

In [ ]:
next(curve_gen)

In [ ]:
# Declare the HoloViews object
start = 0
end = 10
hmap = hv.HoloMap({i: hv.Image(np.random.rand(10,10)) for i in range(start, end+1)})

# Convert the HoloViews object into a plot
plot = renderer.get_plot(hmap)

def animate_update():
    year = slider.value + 1
    if year > end:
        year = start
    slider.value = year

def slider_update(attrname, old, new):
    plot.update(slider.value)

slider = Slider(start=start, end=end, value=0, step=1, title="Year")
slider.on_change('value', slider_update)

def animate():
    if button.label == '► Play':
        button.label = '❚❚ Pause'
        curdoc().add_periodic_callback(animate_update, 200)
    else:
        button.label = '► Play'
        curdoc().remove_periodic_callback(animate_update)

button = Button(label='► Play', width=60)
button.on_click(animate)

# Combine the bokeh plot on plot.state with the widgets
layout = layout([
    [plot.state],
    [slider, button],
], sizing_mode='fixed')

curdoc().add_root(layout)

In [ ]:
show(plot.state)

In [ ]:
pn.panel(button), type(pn.panel(button))

In [ ]:
type(pn.panel(slider))

In [ ]:
n = 100
ps = np.linspace(0,1,num=n)

# define curve function
xfunc = lambda p: np.sin(2*np.pi*p)
yfunc = lambda p: np.cos(2*np.pi*p)
cfunc = lambda p: (xfunc(p), yfunc(p))

In [ ]:
debugbox = pn.widgets.TextInput()
debugbox
    

In [ ]:
gcount = 0
def reset_cb(x):
    gcount += 1
    debugbox.value = x
    

# Continue here!
Modified: Jul 13, 2019 (evening)


In [ ]:
# todo 1:
# Define a button widget (bokeh or panel.widgets object) outside here
# then, link this widget's parameter's callable to a parameterized object's method to set the value of p parameter to 0
reset_button = Button(label='Reset')
show(reset_button)

## but the widget shown by `show` function doesn't actually change the value. Check reset_button.clicks to confirm this.
## this is because we need to add `curdoc().add_root(bt)` and then launch bokeh server: `bokeh serve --show example.py`.
## but I dont want to use bokeh server....:( i just want simple interactivity.... what to do then?


In [ ]:
# todo 2:
# levelset implementation. 
# 1- how to implement affine transformation in discrete space?
## ANS: we need a `warp` function

In [ ]:
def reset_cb(event):
    print(event)
    debugbox.value = str(event)
    

In [ ]:
reset_button.on_click(reset_cb)

In [ ]:
class CurveSimulator(param.Parameterized):

    n_steps = param.Integer(label='Number of simulation steps', default=100)
    p = param.ObjectSelector(label='p', default=0., objects=np.linspace(0,1,num=n_steps.default))
    reset = param.Action(lambda x: x.set_param(p=0.0), doc="Click to clear the buffer and reset p")
#     reset = param.Boolean(default=False,doc="Click to clear the buffer and reset p")


    
    ################################################################################
    # Constant class properties
    ################################################################################
    H,W = 500,500
    curve_opts = opts.Points(size=5,xlim=(-1,1), ylim=(-1,1),width=W, height=H)
    xopts = opts.Points('XCoord', width=W, height=H, size=5, xlim=(0,1), 
                        padding=0.1, invert_axes=True, invert_yaxis=True)
    yopts = opts.Points('YCoord', width=W, height=H, size=5, xlim=(0,1), padding=0.1, invert_xaxis=True)
    
    
    ################################################################################
    # Parameter Dependencies
    ################################################################################    
    @param.depends('n_steps', watch=True)
    def _update_p(self):
        self.count['p'] += 1
        self.param['p'].objects = np.linspace(0,1,num=self.n_steps)
        print('updated p with new number of simulation steps: ', self.n_steps)
    
    @param.depends('p', watch=True)
    def send_point(self):
        point = pd.DataFrame([(self.p, *self.cfunc(self.p))], columns=['p', 'x','y'])
        self.data_src.emit(point)
        time.sleep(0.1)
        
#     @param.depends('reset', watch=True)
#     def reset_handler(self):
#         self.count['reset'] += 1
# #         self.p = 0.0
#         self.set_param(p=0.0)

        
    ################################################################################
    # Initialization
    ################################################################################
    def __init__(self, cfunc, n_steps=100, **kwargs):
        """
        Args:
        - cfunc (function): given an input of a float p in [0,1], returns (x,y), a 
        tuple of x and y coords
        
        - n_steps (int): number of simulation steps along the range of [0,1] for 
        the parameter, p
        """
        super().__init__(**kwargs)
        self.count = defaultdict(int)
        self.cfunc = cfunc 
        self.n_steps = n_steps
        self.curve_generator = gen_curve(self.cfunc, self.n_steps)
        
        self.example = pd.DataFrame({'p': [], 'x':[], 'y':[]})
        self.data_src = streamz.dataframe.DataFrame(example=self.example)
        self.dfstream = Buffer(self.data_src, length=100, index=False)
        self.set_dmap_curve()
        self.set_dmap_x()
        self.set_dmap_y()
        self.overlay = (self.dmap_curve + self.dmap_y + self.dmap_x).cols(2)
    
    def set_curve_generator(self):
        self.curve_generator = gen_curve(self.cfunc, self.n_steps)

    def set_dmap_curve(self):
        self.count['set_curve'] += 1
        dmap_curve = hv.DynamicMap(
            lambda data: hv.Points(data, kdims=['x','y'], group='Curve'),
            streams=[self.dfstream]).opts(color='p')
        self.dmap_curve = dmap_curve.opts(self.curve_opts)
        
    def set_dmap_x(self):
        self.count['set_x'] += 1
        dmap_x = hv.DynamicMap(
            lambda data: hv.Points( data, kdims=['p','x'], group='XCoord'),
            streams=[self.dfstream]).opts(color='p')
        self.dmap_x = dmap_x.opts(self.xopts)
        
    def set_dmap_y(self):
        self.count['set_y'] += 1
        dmap_y = hv.DynamicMap(
            lambda data: hv.Points( data, kdims=['p','y'], group='YCoord'),
            streams=[self.dfstream]).opts(color='p')
        self.dmap_y = dmap_y.opts(self.yopts)
    
    
    ################################################################################
    # Display DynammicMaps
    ################################################################################ 
    def viewable(self):
        return self.overlay
        
    ################################################################################
    # Print Info
    ################################################################################ 
    def get_info(self):
        info = dict()
        info['n_steps']= self.n_steps
        info['example'] = self.example
        info['data_src'] = self.data_src
        info['dfstream'] = self.dfstream.data.describe() if len(self.dfstream.data)  else None
        info['to_go']= self.to_go
        return info 

    
        

In [ ]:
# define curve function
xfunc = lambda p: np.sin(2*np.pi*p)
yfunc = lambda p: np.cos(2*np.pi*p)
cfunc = lambda p: (xfunc(p), yfunc(p))
c = CurveSimulator(cfunc)

In [ ]:
# pp(c.get_info())

In [ ]:
pn.Row(
    pn.Param(c.param, widgets={
    'p': pn.widgets.DiscretePlayer,
    'reset': pn.widgets.Button
        
        }, width=500),
    pn.panel(c.viewable())
)
# pn.Param(c.param)


---
Modified: Jul 13, 2019
## Button widget

In [ ]:
button = pn.widgets

In [ ]:
pslider = Slider(start=0., end=1., value=0., step=0.01)

In [ ]:
show(pslider)

In [ ]:
getattr?

In [ ]:
c.get_param_values('p')

In [ ]:
c.p

In [ ]:
class Temp(param.Parameterized):
    p = param.Number(0., bounds=(0., 1.))
    reset = param.Action(lambda x: x.set_param(p=0.5))
    

In [ ]:
t = Temp()
t.p, t.reset

In [ ]:
pn.Param(t.param)